In [1]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked fro

In [2]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100)#,500,1000)
time_off<-c(10000,10,20)#,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [3]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [5]:
m<-1
list_of_vi=list()
corpus_list<-c("google")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("UnTagged")

ppmi_setting_list<-c("PPMI")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("woSetting","withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [6]:
for (c in corpus_list){
for (t in tagged_list) {
  for (f in feature_setting_list) {
  for (p in ppmi_setting_list){
    for (a in comp_setting_list){
      for (i in time_off) {
        
        for (j in cut_off) {
          

            print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
            
            na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
            nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
            
            for (r in ratings_list){
              na_df<-na_df %>% filter(source==r)
              nona_df<-nona_df %>% filter(source==r)
              
              trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
              trainY_na<-na_df %>% select(compositionality)
              trainY_na<-trainY_na$compositionality
              
              
              trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
              trainY_nona<-nona_df %>% select(compositionality)
              trainY_nona<-trainY_nona$compositionality
              
              if (dim(trainX_nona)[1]<10) {
                break
              }
              for (s in seed_list)  {
                set.seed(s)
                seeds <- vector(mode = "list", length = 14)
                for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                #for the last model
                seeds[[14]]<-sample.int(1000, 1)
                
                ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                    preProc = c("nzv","center", "scale")
                )
                
                elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                )
                
                lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                    preProc = c("nzv","center", "scale")
                )
                
                
                ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                  trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                  preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                  trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                  preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                
                ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                             preProc = c("nzv","center", "scale")
                )
                
                elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                )
                
                lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                             preProc = c("nzv","center", "scale")
                )
                
                
                ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                           trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                           preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                           trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                           preProcess = c("nzv","medianImpute", "center", "scale")
                )
                

                perf_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                perf_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                perf_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                
                
                perf_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                perf_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                perf_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                
                
                list_of_rsqr[[m]]<-perf_elastic_na
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_elastic_nona
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_ridge_na
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_ridge_nona
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_lasso_na
                m<-m+1 
                
                list_of_rsqr[[m]]<-perf_lasso_nona
                m<-m+1 
                
            }
            

          
            

              
              #varimp_elastic<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="elastic",seed=s,timespan=i,cutoff=j,t(varImp(elastic)$importance),ppmi=p)
              #varimp_ridge<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="ridge",seed=s,timespan=i,cutoff=j,t(varImp(ridge)$importance),ppmi=p)
              #varimp_lasso<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="lasso",seed=s,timespan=i,cutoff=j,t(varImp(lasso)$importance),ppmi=p)
              #varimp_rpart<-data.frame(tag=t,setting=a,dataset=k,corpus="coha",ml_algo="rpart",seed=s,timespan=i,cutoff=j,t(varImp(rpart)$importance))
              
              #varimp_elastic_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="elastic",seed=s,timespan=i,cutoff=j,t(varImp(elastic_nona)$importance),ppmi=p)
              #varimp_ridge_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="ridge",seed=s,timespan=i,cutoff=j,t(varImp(ridge_nona)$importance),ppmi=p)
              #varimp_lasso_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="lasso",seed=s,timespan=i,cutoff=j,t(varImp(lasso_nona)$importance),ppmi=p)    
              #varimp_rpart_nona<-data.frame(tag=t,setting=a,dataset=k,corpus="coha",ml_algo="rpart_nona",seed=s,timespan=i,cutoff=j,t(varImp(rpart_nona)$importance))
              

              
              #list_of_rsqr[[m]]<-perf_rpart
              #list_of_vi[[m]]<-varimp_rpart
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpart_nona
              #list_of_vi[[m]]<-varimp_rpart_nona
              #m<-m+1
              
              
              #list_of_rsqr[[m]]<-perf_elastic_google
              #list_of_vi[[m]]<-varimp_elastic_google
              #m<-m+1
              
              #list_of_rsqr[[m]]<-perf_ridge_google
              #list_of_vi[[m]]<-varimp_ridge_google
              #m<-m+1
              
              #list_of_rsqr[[m]]<-perf_lasso_google
              #list_of_vi[[m]]<-varimp_lasso_google
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpart_google
              #list_of_vi[[m]]<-varimp_rpart_google
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpartnona_google
              #list_of_vi[[m]]<-varimp_rpartnona_google
              #m<-m+1 
            }
          
        }
      }
    }
  }
  }
}
}

[1] "google PPMI Aware woSetting UnTagged 10000 0"
[1] "google PPMI Aware woSetting UnTagged 10000 10"
[1] "google PPMI Aware woSetting UnTagged 10000 50"
[1] "google PPMI Aware woSetting UnTagged 10000 100"
[1] "google PPMI Aware woSetting UnTagged 10 0"
[1] "google PPMI Aware woSetting UnTagged 10 10"
[1] "google PPMI Aware woSetting UnTagged 10 50"
[1] "google PPMI Aware woSetting UnTagged 10 100"
[1] "google PPMI Aware woSetting UnTagged 20 0"
[1] "google PPMI Aware woSetting UnTagged 20 10"
[1] "google PPMI Aware woSetting UnTagged 20 50"
[1] "google PPMI Aware woSetting UnTagged 20 100"
[1] "google PPMI Agnostic woSetting UnTagged 10000 0"


In [17]:
varimp_df<-bind_rows(list_of_vi)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0

In [7]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)

In [9]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
644,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1771,20,50,PPMI,glmnet,0.5279124,0.7313636
476,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,100,PPMI,glmnet,0.5225827,0.7200233
356,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,10,PPMI,glmnet,0.5231564,0.7131652
175,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,50,PPMI,glmnet,0.5585807,0.7128805
177,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,50,PPMI,glmnet,0.5399065,0.7128805
422,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1001,10,100,PPMI,glmnet,0.5311879,0.7118798
235,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,100,PPMI,glmnet,0.5597754,0.7109973
55,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,0,PPMI,glmnet,0.5582943,0.7095471
57,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,0,PPMI,glmnet,0.5399696,0.7095471


In [10]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
440,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,100,PPMI,glmnet,0.6028381,0.6806244
620,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,50,PPMI,glmnet,0.5885712,0.6776766
320,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,10,PPMI,glmnet,0.5854357,0.6588983
380,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,50,PPMI,glmnet,0.5837606,0.6813442
324,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,10,PPMI,glmnet,0.5819766,0.6440176
444,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,100,PPMI,glmnet,0.5814872,0.6782434
624,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,20,50,PPMI,glmnet,0.5751432,0.6693253
384,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,50,PPMI,glmnet,0.5744349,0.6491037
680,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,100,PPMI,glmnet,0.5685864,0.6998218


In [19]:
write.csv(rsquared_df,"rsquared_Tagged_RAW_Agnostic.csv",row.names = FALSE)

In [20]:
write.csv(varimp_df,"varimp_Tagged_RAW_Agnostic.csv",row.names = FALSE)